In [13]:
import pandas as pd
import numpy as np
nfolds = 10

In [22]:
test_acc = []
froot = 'weapon'
#yes = '{recidivate-within-two-years:Yes}'
yes = '{weapon:yes}'
for i in range(nfolds):
    fname = '../jmlr/for-{0}_{1}_train.out-curious_lb-with_prefix_perm_map-minor-removed=none-max_num_nodes=1000000000-c=0.0100000-v=10-f=1000-opt.txt'.format(froot, i)
    with open(fname, 'r') as f:
        line = f.readline()
        opt = map(lambda x: x.split('~'), line.split(';'))
        #opt = map(lambda x: (x[0], int(x[1])), opt)
        #opt = line.split(';')
        print opt
        
    nrules = 0
    with open('../data/CrossValidation/{0}_{1}_test.out'.format(froot, i)) as f:
        line = f.readline()
        nrules = len(line.split()) - 1
    out = pd.read_csv('../data/CrossValidation/{0}_{1}_test.out'.format(froot, i), sep=' ', names=['Rule'] + range(nrules))
    label = pd.read_csv('../data/CrossValidation/{0}_{1}_test.label'.format(froot, i), sep=' ', names=['Rule'] + range(nrules))
    out.set_index('Rule', inplace=True)
    label.set_index('Rule', inplace=True)
    
    captured = set()
    preds = []
    for (rule, pred) in opt[:-1]:
        cappd = out.ix[rule] == 1
        unfiltered_cappd = out.T[cappd].index
        filtered_cappd = unfiltered_cappd.difference(captured)
        preds += map(lambda x: (x, pred), filtered_cappd)
        captured = captured.union(filtered_cappd)

    # Handle default rule
    _, def_pred = opt[-1]
    preds += [(x, def_pred) for x in set(range(nrules)).difference(captured)]
    sorted_preds = sorted(preds, key=lambda x: x[0])
    pred_arr = pd.Series(map(lambda x: int(x[1]), sorted_preds))
    true_arr = label.loc[yes]

    cnf_mat = pd.crosstab(pred_arr, true_arr)
    true_positive = cnf_mat[0][0]
    false_positive = cnf_mat[1][0]
    false_negative = cnf_mat[0][1]
    true_negative = cnf_mat[1][1]
    corr = float(true_positive + true_negative)

    test_acc.append(corr / float(nrules))
print test_acc

[['{location:transit-authority}', '1'], ['{cs_objcs:stop-reason=suspicious-object}', '1'], ['{cs_bulge:stop-reason=suspicious-bulge}', '1'], ['default', '0']]
[['{cs_objcs:stop-reason=suspicious-object}', '1'], ['{location:transit-authority}', '1'], ['{cs_bulge:stop-reason=not-suspicious-bulge}', '0'], ['default', '1']]
[['{location:transit-authority}', '1'], ['{cs_objcs:stop-reason=suspicious-object}', '1'], ['{cs_bulge:stop-reason=not-suspicious-bulge}', '0'], ['default', '1']]
[['{cs_objcs:stop-reason=suspicious-object}', '1'], ['{location:transit-authority}', '1'], ['{cs_bulge:stop-reason=not-suspicious-bulge}', '0'], ['default', '1']]
[['{location:transit-authority}', '1'], ['{cs_bulge:stop-reason=suspicious-bulge}', '1'], ['{cs_objcs:stop-reason=suspicious-object}', '1'], ['default', '0']]
[['{location:transit-authority}', '1'], ['{cs_bulge:stop-reason=suspicious-bulge}', '1'], ['{cs_objcs:stop-reason=suspicious-object}', '1'], ['default', '0']]
[['{cs_objcs:stop-reason=suspiciou

In [17]:
np.std(test_acc)

0.01758652738768542

In [ ]:
fname = '../logs/for-bcancer.out-curiosity-with_prefix_perm_map-max_num_nodes=100000-c=0.0010000-v=1-f=1000-opt.txt'
test_acc = []
for i in range(nfolds):
    fname = '../jmlr/for-compas_{0}_train.out-curious_lb-with_prefix_perm_map-minor-removed=none-max_num_nodes=1000000000-c=0.0050000-v=10-f=1000-opt.txt'.format(i)
    with open(fname, 'r') as f:
        line = f.readline()
        opt = map(lambda x: x.split('~'), line.split(';'))
        #opt = map(lambda x: (x[0], int(x[1])), opt)
        #opt = line.split(';')
        print opt
        
    nrules = 0
    with open('../data/CrossValidation/compas_{0}_test.out'.format(i)) as f:
        line = f.readline()
        nrules = len(line.split()) - 1
    out = pd.read_csv('../data/CrossValidation/compas_{0}_test.out'.format(i), sep=' ', names=['Rule'] + range(nrules))
    label = pd.read_csv('../data/CrossValidation/compas_{0}_test.label'.format(i), sep=' ', names=['Rule'] + range(nrules))
    out.set_index('Rule', inplace=True)
    label.set_index('Rule', inplace=True)
    
    captured = set()
    preds = []
    for (rule, pred) in opt[:-1]:
        cappd = out.ix[rule] == 1
        unfiltered_cappd = out.T[cappd].index
        filtered_cappd = unfiltered_cappd.difference(captured)
        preds += map(lambda x: (x, pred), filtered_cappd)
        captured = captured.union(filtered_cappd)

    # Handle default rule
    _, def_pred = opt[-1]
    preds += [(x, def_pred) for x in set(range(nrules)).difference(captured)]
    sorted_preds = sorted(preds, key=lambda x: x[0])
    pred_arr = pd.Series(map(lambda x: int(x[1]), sorted_preds))
    true_arr = label.loc['{recidivate-within-two-years:Yes}']

    cnf_mat = pd.crosstab(pred_arr, true_arr)
    true_positive = cnf_mat[0][0]
    false_positive = cnf_mat[1][0]
    false_negative = cnf_mat[0][1]
    true_negative = cnf_mat[1][1]
    corr = float(true_positive + true_negative)

    test_acc.append(corr / float(nrules))
print test_acc